# Stochastic Analysis of a simple recombinase-based oscillator

In this Jupyterbook, we implement the Gilleaspy algorithm of a recombinase-based oscillator at a single copy 

In [1]:
# Colab setup
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
  
    from google.colab import drive
    drive.mount('/content/drive')
    dir = "/content/drive/My Drive/Research/2022/RecombinaseOscillator/SimpleRecombinase/"
else:
    dir = "/Users/christian/My Drive/Research/2022/RecombinaseOscillator/SimpleRecombinase/"
# ------

# This check the number of processors in the computer
#% cat /proc/cpuinfo | grep processor | wc -l

# % pip install biocircuits bokeh-catplot watermark blackcellmagic
import multiprocessing
import tqdm

import numpy as np
import scipy.stats as st
import numba
import math

from scipy.signal import argrelextrema
import scipy.fftpack
import os.path

import biocircuits

# Plotting modules
import bokeh.io
import bokeh.plotting
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.io import export_svgs
from bokeh.models import Range1d

bokeh.io.output_notebook()

Loading BokehJS ...

In [14]:
def JoinMatrices(A,B):
    n1, m1 = A.shape
    n2, m2 = B.shape
    ZA = np.zeros((n1,m2))
    ZB = np.zeros((n2,m1))
    tmp1 = np.hstack((A,ZA))
    tmp2 = np.hstack((ZB,B)) 
    return np.vstack((tmp1,tmp2))

In [15]:
# Stoichiometry  matrix of a single switch
# R00, R10, R01, R11
S1 = np.array(
    [
     [-1, 1, 0, 0],  # R00 -> R10
     [ 1, -1, 0, 0], # R10 -> R00 
     [ 0, -1, 0, 1], # R10 -> R11
     [ 0, 1, 0, -1], # R11 -> R10

     [-1, 0, 1, 0],  # R00 -> R01
     [ 1, 0, -1, 0], # R01 -> R00 
     [ 0, 0, -1, 1], # R01 -> R11
     [ 0, 0, 1, -1], # R11 -> R01 

     [ 0, 0,  0, -1], # R11 -> L00
    ],
    dtype=int,
)
# Stoichiometry  matrix of two switches
S2 = JoinMatrices(S1,S1)
S2[8,4] = 1
S2[17,0] = 1
#print(S2)

In [16]:
# Stoichiometry  matrix of transcription and translation
# m1, x1, y1, m2, x2, y2, c
S3 = np.array(
    [
     [ 1, 0, 0, 0, 0, 0, 0], # m1 production (TX)
     [-1, 0, 0, 0, 0, 0, 0], # m1 decay
     [ 0, 1, 0, 0, 0, 0, 0], # x1 production (TL)
     [ 0,-1, 0, 0, 0, 0, 0], # x1 decay
     [ 0,-2, 1, 0, 0, 0, 0], # y1 production (dimerization)
     [ 0, 2,-1, 0, 0, 0, 0], # y1 dissociation
     [ 0, 0,-1, 0, 0, 0, 0], # y1 decay

     [ 0, 0, 0, 1, 0, 0, 0], # m2 production (TX)
     [ 0, 0, 0,-1, 0, 0, 0], # m2 decay
     [ 0, 0, 0, 0, 1, 0, 0], # x2 production (TL)
     [ 0, 0, 0, 0,-1, 0, 0], # x2 decay
     [ 0, 0, 0, 0,-2, 1, 0], # y2 production (dimerization)
     [ 0, 0, 0, 0, 2,-1, 0], # y2 dissociation
     [ 0, 0, 0, 0, 0,-1, 0], # y2 decay

     [ 0,-1, 0, 0,-1, 0, 1], # c production (heterodimer formation)
     [ 0, 1, 0, 0, 1, 0,-1], # c heterodimer dissociation
     [ 0, 0, 0, 0, 0, 0,-1], # c decay
    ],
    dtype=int,
)
# Stoichiometry  matrix of two switches
simple_update = JoinMatrices(S2,S3)
# Updating the consumption of recombinase in the switches
S0 = np.array(
    [
     [-1, 1, -1, 1,-1, 1, -1, 1, 2],  # R00 -> R10
    ],
    dtype=int,
)
simple_update[0:9,10] = S0
simple_update[9:18,13] = S0

#print(simple_update)
simple_update.shape

(35, 15)

In [17]:
def simple_propensity(propensities, x, t, th, rh, ph, d, g, gd, a0, d0, r, Omega, n):
    #x, y, c = population
    # Switch 1 (Promoter pointing to the right R)
    propensities[0] = a0 * x[0] * x[10]  / Omega # R00 -> R10
    propensities[1] = d0 * x[1] # R10 -> R00
    propensities[2] = a0 * x[1] * x[10]  / Omega # R10 -> R11
    propensities[3] = d0 * x[3] # R11 -> R10
    propensities[4] = a0 * x[0] * x[10]  / Omega # R00 -> R01
    propensities[5] = d0 * x[2] # R01 -> R00
    propensities[6] = a0 * x[2] * x[10]  / Omega # R01 -> R11
    propensities[7] = d0 * x[3] # R11 -> R01
    propensities[8] = r * x[3] # R11 -> L00

    # Switch 2 (Promoter pointing to the left L)
    propensities[ 9] = a0 * x[4] * x[13]  / Omega # L00 -> L10
    propensities[10] = d0 * x[5] # L10 -> L00
    propensities[11] = a0 * x[5] * x[13]  / Omega # R10 -> R11
    propensities[12] = d0 * x[7] # R11 -> R10
    propensities[13] = a0 * x[4] * x[13]  / Omega # R00 -> R01
    propensities[14] = d0 * x[6] # R01 -> R00
    propensities[15] = a0 * x[6] * x[13]  / Omega # R01 -> R11
    propensities[16] = d0 * x[7] # R11 -> R01
    propensities[17] = r * x[7] # R11 -> L00

    # TX-TL of recombinase 1
    propensities[18] = th * x[0] # R00 -> R00 + M1
    propensities[19] = ph * x[8] # M1 -> 0
    propensities[20] = rh * x[8] # M1 -> M1 + X1
    propensities[21] = d * x[9] # X1 -> 0
    propensities[22] = g * x[9] * (x[9]-1)  / 2 / Omega # X1 + X1 -> Y1
    propensities[23] = gd * x[10] # Y1 -> X1 + X1
    propensities[24] = d * x[10] # Y1 -> 0

    # TX-TL of recombinase 2
    propensities[25] = th * x[4] # L00 -> L00 + M2
    propensities[26] = ph * x[11] # M2 -> 0
    propensities[27] = rh * x[11] # M2 -> M2 + X2
    propensities[28] = d * x[12] # X2 -> 0
    propensities[29] = g * x[12] * (x[12]-1)  / 2 / Omega # X2 + X2 -> Y2
    propensities[30] = gd * x[13] # Y2 -> X2 + X2
    propensities[31] = d * x[13] # Y2 -> 0

    # Heterodimmer reaction
    propensities[32] = g * x[9] * x[12] / Omega # L00 -> L00 + M2
    propensities[33] = gd * x[14] # M2 -> 0
    propensities[34] = d * x[14] # M2 -> M2 + X2                   

In [34]:
# Specify parameters for calculation
n = 1 # copy number of promoter
th = .3*60/n # 1/h
rh = 80 # 1/h
ph = 0.6931/10*60 # 1/h log(2)=0.6931
d = 0.6931/30*60 # 1/h log(2)=0.6931
g = 2.1*36 # 1/uM/h 
gd = g*0.01 # 1/h
r = 5 # 1/h Switching -- rate 5 and 1 for paper
a0 = 2.1*36
d0 = 0.1*a0

Omega = 600 # 600

args = (th, rh, ph, d, g, gd, a0, d0, r, Omega, n)
time_points = np.linspace(0, 40, 401)
population_0 = np.zeros(15, dtype=int)
population_0[0] = n

samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=1, args=args, n_threads=1,progress_bar=False)

# Set up plots
fig_size = [300, 200] # for paper size (260/115), half size (144,115), visualize (600/200)
x_range = (0, time_points[-1])
y_range = (0, 150)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range, y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

# Plot trajectories of recombinase 1 and 2
y1 = samples[0,:,10] # samples[:,:,10][0]
plots[0].step(time_points, y1, line_width=2, alpha= 1, line_join="bevel",color="grey")

y2 = samples[0,:,13]
plots[0].step(time_points, y2, line_width=2, alpha= 1, line_join="bevel",color="orange")
# Link axes

bokeh.io.show(bokeh.layouts.row(plots))

#plots[0].output_backend = "svg"
#export_svgs(plots,filename="fig/SingleTrajectory.svg")

In [35]:
# Set up plots
# For two digits 260 by 115 larger and 144 by 115 for half size
# For three digits 266 by 115 larger and 150 by 115 for half size
fig_size = [266, 115] # visualize (600/200)
x_range = (0, 40)
y_range = (0, 100)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

# Plot trajectories of recombinase 1 and 2
sR = samples[0,:,4:8].sum(axis=1)/n
#print(sR.size)

plots[0].varea(time_points, sR*0, sR*y_range[1], fill_color="#fef2e9")

y1 = samples[0,:,10] # samples[:,:,10][0]
plots[0].step(time_points, y1, line_width=2, alpha= 0.5, line_join="bevel",color="#999999")

y2 = samples[0,:,13]
plots[0].step(time_points, y2, line_width=2, alpha= 1, line_join="bevel",color="#f48024")
# Link axes

bokeh.io.show(bokeh.layouts.row(plots))

plots[0].output_backend = "svg"
export_svgs(plots,filename="fig/SingleTrajectory.svg")

['fig/SingleTrajectory.svg']

In [7]:
# Computing the auto-correlation

def correlate(x, y, dt):
    # Compute correlation with max correlation being unity
    corr = np.correlate(x, y, "full")
    corr /= corr.max()

    idx = np.where(corr==1.)[0][0]
    corr = corr[idx:]
    lags = np.arange(0, len(corr))*dt
    return lags, corr

In [ ]:
fig_size = [260, 115] # for paper size (260/115), half size (144,115), visualize (600/200)
x_range = (0, time_points[-1])
y_range = (0, 200)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, 
                          #y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

dt = time_points[1] - time_points[0]
y1 = samples[0,:,10] # samples[:,:,10][0]
y1s = y1 - y1.mean()
lags, auto_corr = correlate(y1s, y1s, dt)
max_id = argrelextrema(auto_corr, np.greater)

plots[0].step(time_points, y1, line_width=2, alpha= 1, line_join="bevel",color="grey")
#plots[0].line(lags, auto_corr*130, line_width=2, alpha= 1, line_join="bevel",color="black")

plots[1].line(lags, auto_corr, line_width=2, alpha= 1, line_join="bevel",color="grey")
plots[1].circle(lags[max_id], auto_corr[max_id], line_width=2, alpha= 1, line_join="bevel",color="red")

y1 = samples[0,:,13] # samples[:,:,10][0]
y1s = y1 - y1.mean()
lags, auto_corr = correlate(y1s, y1s, dt)
max_id = argrelextrema(auto_corr, np.greater)

plots[0].step(time_points, y1, line_width=2, alpha= 1, line_join="bevel",color="orange")
#plots[0].line(lags, auto_corr*130, line_width=2, alpha= 1, line_join="bevel",color="black")
plots[1].line(lags, auto_corr, line_width=2, alpha= 1, line_join="bevel",color="orange")
plots[1].circle(lags[max_id], auto_corr[max_id], line_width=2, alpha= 1, line_join="bevel",color="black")

#bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=2))
bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
def FFT(t1,x1,fr):
    n = len(x1)
    if n % 2:
        x = x1[:-2]
        t = t1[:-2]
        n = len(x)
    else:
        x = x1[:-1]
        t = t1[:-1]

    dt = (t[1]-t[0])
    tf = scipy.fftpack.fftfreq(x.size, d=dt)
    xf = scipy.fftpack.fft(x)
    xf = np.abs(xf)**2
    idx = np.where((tf >= fr[0]) & (tf <=fr[1]))
    return tf[idx],xf[idx]

In [ ]:
fig_size = [150, 115] # for paper size (260/115), half size (144,115), visualize (600/200)
x_range = (0, time_points[-1])
y_range = (0, 200)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

dt = time_points[1] - time_points[0]

y1 = samples[0,:,10] # samples[:,:,10][0]
y1s = y1 - y1.mean()
lags, auto_corr = correlate(y1s, y1s, dt)
f,Mf = FFT(lags, auto_corr, (0, 1))

plots[0].line(lags, auto_corr, line_width=2, alpha= 1, line_join="bevel",color="grey")
plots[1].line(f, Mf, line_width=2, alpha= 1, line_join="bevel",color="grey")

y1 = samples[0,:,13] # samples[:,:,10][0]
y1s = y1 - y1.mean()
lags, auto_corr = correlate(y1s, y1s, dt)

f,Mf = FFT(lags, auto_corr, (0, 1))
#f,Mf = FFT(time_points, y1s, (0, 1))

plots[0].line(lags, auto_corr, line_width=2, alpha= 1, line_join="bevel",color="orange")
plots[1].line(f, Mf, line_width=2, alpha= 1, line_join="bevel",color="orange")

plots[1].output_backend = "svg"
export_svgs(plots[1],filename="fig/Plot_freq.svg")
bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
# Specify parameters for calculation
n = 1 # copy number of promoter
th = .3*60/n # 1/h
rh = 80 # 1/h
ph = 0.6931/10*60 # 1/h log(2)=0.6931
d = 0.6931/30*60 # 1/h log(2)=0.6931
g = 2.1*36 # 1/uM/h 
gd = g*0.01 # 1/h
r = 1 # 1/h Switching -- rate 5 and 1 for paper
a0 = 2.1*36
d0 = 0.1*a0

Omega = 600 # 600
size = 125
#dir = "/content/drive/My Drive/Research/2022/RecombinaseOscillator/"
f_name = dir+"Model_1_Data_"+str(size)+"_r"+str(r)+".npy"

args = (th, rh, ph, d, g, gd, a0, d0, r, Omega, n)
time_points = np.linspace(0, 40, 401)
population_0 = np.zeros(15, dtype=int)
population_0[0] = n

if os.path.exists(f_name):
    samples = np.load(f_name)
else:
    samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=size, args=args, n_threads=4,progress_bar=True)
    #np.savetxt(f_name, samples, fmt='%d')
    np.save(f_name, samples)


In [ ]:
# This function finds the time of the peak
def FindPicks(t,samples):
    ys = samples - samples.mean()
    lags, auto_corr = correlate(ys, ys, t[1]-t[0])
    max_id = argrelextrema(auto_corr, np.greater)
    return t[max_id[0][0:4]]

In [ ]:
t_peaks = FindPicks(time_points,samples[0,:,10])
for i in range(1,samples.shape[0]):
    t_peaks = np.vstack((t_peaks,FindPicks(time_points,samples[i,:,10])))

#print(tmp)

In [ ]:
fig_size = [300, 200] # for paper size (144, 110) # for larger size (144,144)
x_range = (0, time_points[-1])
y_range = (0, 200)

plots = []
for i in range(3):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

x = np.linspace(0,20,41)

for j in range(3):
    hist, bin_edges = np.histogram(t_peaks[:,j], x, density=True)
    plots[j].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:],fill_color="grey", line_color="grey", fill_alpha=0.2,line_alpha=0.)
    plots[j].step(bin_edges[:-1],hist, line_width=2, mode="after", color = "grey")

bokeh.io.show(bokeh.layouts.row(plots))

In [8]:
# This function finds the Delta T between peak times
def FindDeltaT(t,samples):
    ys = samples - samples.mean()
    lags, auto_corr = correlate(ys, ys, t[1]-t[0])
    max_id = argrelextrema(auto_corr, np.greater)
    return t[max_id[0][1:5]]-t[max_id[0][0]]

In [ ]:
Delta_T = FindDeltaT(time_points,samples[0,:,10])
for i in range(1,samples.shape[0]):
    tmp = FindDeltaT(time_points,samples[i,:,10])
    if len(tmp)>3:
        Delta_T = np.vstack((Delta_T,tmp))

In [ ]:
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
x_range = (0, 40)
y_range = (0, 140)

plots = []
for i in range(4):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

x = np.linspace(0,50,51)

for j in range(4):
    hist, bin_edges = np.histogram(Delta_T[:,j], x, density=False)
    plots[j].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:],fill_color="grey", line_color="grey", fill_alpha=0.2,line_alpha=0.)
    plots[j].step(bin_edges[:-1],hist, line_width=2, mode="after", color = "grey")

bokeh.io.show(bokeh.layouts.row(plots))

#### Computing the peak vs the variance

In [ ]:
# Set up plots
# For two digits 260 by 115 larger and 144 by 115 for half size
# For three digits 266 by 115 larger and 150 by 115 for half size
fig_size = [144, 115] # for paper size (144, 110) # for larger size (144,144)
x_range = (0, 5)
y_range = (0, 40)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

vx = range(1,5)
vy = np.var(Delta_T, axis=0)

plots[0].circle(vx,vy, line_width=2,color="black")

L_eq = np.polyfit(vx,vy,1)
plots[0].line(vx, np.poly1d(L_eq)(vx), line_width=2, alpha= 0.5, line_join="bevel",color="blue")

plots[0].output_backend = "svg"
#export_svgs(plots[0],filename="fig/Plot_IncoherentMetric_r1.svg")

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
L_eq

In [ ]:
# Specify parameters for calculation
n = 1 # copy number of promoter
th = .3*60/n # 1/h
rh = 80 # 1/h
ph = 0.6931/10*60 # 1/h log(2)=0.6931
d = 0.6931/30*60 # 1/h log(2)=0.6931
g = 2.1*36 # 1/uM/h 
gd = g*0.01 # 1/h
r = 5 # 1/h Switching rate
a0 = 2.1*36/2
d0 = 0.1*a0

Omega = 600 # 600
size = 250
state = 10 # 10 (recombinase 1) or 13 (recombinase 2)
args = (th, rh, ph, d, g, gd, a0, d0, r, Omega, n)

time_points = np.linspace(0, 40, 401) # 40 by 401
population_0 = np.zeros(15, dtype=int)
population_0[0] = n

param = (2,3,4,6,8)
#param = range(0,8)
scale = (1./4, 1./3, 1./2, 1., 2.,3., 4.)
vx = range(1,5)

out = np.array((0., 1./4, 1./3, 1./2, 1., 2., 3., 4.))

#dir = "/content/drive/My Drive/Research/2022/RecombinaseOscillator/"
f_name2 = dir+"Model_1_AllSensitivity_"+str(size)+".npy"

if os.path.exists(f_name2):
    out = np.load(f_name2)
    idx = param.index(out[-1,0])+1
else:
    idx = 0

if idx<len(param):
    for par in param[idx:]:
        print(par)
        tmp = np.array((par))
        args_i = np.array((th, rh, ph, d, g, gd, a0, d0, r, Omega, n))
        for fac in scale:
            # Update model para
            args_i[par] = args[par]*fac
            samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=size, args=tuple(args_i), n_threads=2,progress_bar=False)

            # Stack T1, T2, T3, T4
            Delta_T = FindDeltaT(time_points,samples[0,:,state])
            for i in range(1,samples.shape[0]):
                Delta_T = np.vstack((Delta_T,FindDeltaT(time_points,samples[i,:,state])))

            # Compute variance and find slope (incoherent metric)
            vy = np.var(Delta_T, axis=0)
            L_eq = np.polyfit(vx,vy,1)
            tmp = np.append(tmp,L_eq[0])
    
        out = np.vstack((out,tmp))
        np.save(f_name2, out)

print(out)

In [12]:
# Specify parameters for calculation
n = 3 # copy number of promoter
th = .3*60/n # 1/h
rh = 80 # 1/h
ph = 0.6931/10*60 # 1/h log(2)=0.6931
d = 0.6931/30*60 # 1/h log(2)=0.6931
g = 2.1*36 # 1/uM/h 
gd = g*0.01 # 1/h
r = 5 # 1/h Switching rate
a0 = 2.1*36
d0 = 0.1*a0

Omega = 600 # 600
size = 500
state = 13 # 10 (recombinase 1) or 13 (recombinase 2)
args = (th, rh, ph, d, g, gd, a0, d0, r, Omega, n)

time_points = np.linspace(0, 40, 401) # 40 by 401
population_0 = np.zeros(15, dtype=int)
population_0[0] = n

param = (1,3,4,6,8)
#param = range(0,8)
scale = (1./4, 1./3, 1./2, 1., 2.,3., 4.)
vx = range(1,5)

out = np.array((0., 1./4, 1./3, 1./2, 1., 2., 3., 4.))

#dir = "/content/drive/My Drive/Research/2022/RecombinaseOscillator/"
f_name2 = dir+"Model_1_AllSensitivity_"+str(size)+"_n"+str(n)+".npy"

if os.path.exists(f_name2):
    out = np.load(f_name2)
    idx = param.index(out[-1,0])+1
else:
    idx = 0

if idx<len(param):
    for par in param[idx:]:
        print(par)
        tmp = np.array((par))
        args_i = np.array((th, rh, ph, d, g, gd, a0, d0, r, Omega, n))
        for fac in scale:
            # Update model para
            args_i[par] = args[par]*fac
            ii = 0
            Delta_T = np.array((0.,0.,0.,0.))
            while ii<size+1:
                samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=1, args=tuple(args_i), n_threads=1,progress_bar=False)
                try:
                    tmp_0 = FindDeltaT(time_points,samples[0,:,state])
                    if len(tmp_0) == 4:
                        Delta_T = np.vstack((Delta_T,tmp_0[0:4]))
                        ii = ii + 1
                except: tmp_0 = 0
                            
            # Compute variance and find slope (incoherent metric)
            vy = np.var(Delta_T[1:,:], axis=0)
            L_eq = np.polyfit(vx,vy,1)
            tmp = np.append(tmp,L_eq[0])
    
        out = np.vstack((out,tmp))
        np.save(f_name2, out)

print(out)

1
3
4
6
8
[[0.         0.25       0.33333333 0.5        1.         2.
  3.         4.        ]
 [1.         1.67748861 2.31762609 3.21229038 2.64050158 1.46404587
  0.95099056 0.77965525]
 [3.         4.51585053 3.74035276 3.2869669  2.84624078 1.25322502
  0.69176513 0.32461236]
 [4.         3.85247296 3.43841488 2.9381501  2.3879254  2.62724623
  2.41906839 2.00545047]
 [6.         4.02047558 3.77023856 3.76707312 2.58820985 1.66314941
  1.36852011 1.17373313]
 [8.         2.43209368 2.50235192 3.0545894  2.34051059 1.87216358
  1.8941201  1.51473735]]


In [ ]:
i

In [10]:
# Set up plots
# For two digits 260 by 115 larger and 144 by 115 for half size
# For three digits 266 by 115 larger and 150 by 115 for half size
fig_size = [144, 115] # for paper size (260/115), half size (144,115), visualize (600/200)

x_range = (0, 4.2)
y_range = (0, 20)

plots = []
for i in range(out.shape[0]-1):
    #print(i)
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

color="#f48024"    
for i in range(out.shape[0]-1):
    plots[i].line(out[0,1:], out[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color=color)
    plots[i].circle(out[0,1:], out[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color=color)
    #plots[i].output_backend = "svg"
    #export_svgs(plots[i],filename="fig/Sensitivity_Param_" + str(out[i+1,0]) + "_n.svg")

bokeh.io.show(bokeh.layouts.row(plots))

In [13]:
size = 500
f_name2 = dir+"Model_1_AllSensitivity_"+str(size)+"_NoSeq.npy"
out0 = np.load(f_name2)

f_name2 = dir+"Model_1_AllSensitivity_"+str(size)+".npy"
out1 = np.load(f_name2)

f_name2 = dir+"Model_1_AllSensitivity_"+str(size)+"_n2.npy"
out2 = np.load(f_name2)

f_name2 = dir+"Model_1_AllSensitivity_"+str(size)+"_n3.npy"
out3 = np.load(f_name2)


fig_size = [144, 115] # for paper size (260/115), half size (144,115), visualize (600/200)

x_range = (0, 4.2)
y_range = (0, 14)

plots = []
for i in range(out0.shape[0]-1):
    #print(i)
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

    
for i in range(out0.shape[0]-1):
    plots[i].line(out0[0,1:], out0[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color="grey") # No Seq
    plots[i].circle(out0[0,1:], out0[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color="grey")
    
    plots[i].line(out1[0,1:], out1[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color="#f48024") # orange
    plots[i].circle(out1[0,1:], out1[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color="#f48024") # n=1
    
    plots[i].line(out2[0,1:], out2[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color="#178ccb") # blue
    plots[i].circle(out2[0,1:], out2[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color="#178ccb") # n=2
    
    plots[i].line(out3[0,1:], out3[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color="#716fb2") # purple
    plots[i].circle(out3[0,1:], out3[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color="#716fb2") # n=3

    #plots[i].output_backend = "svg"
    #export_svgs(plots[i],filename="fig/Sensitivity_Param_" + str(out[i+1,0]) + "_n.svg")

bokeh.io.show(bokeh.layouts.row(plots))

#### 